In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import os

from predict_policing_agent import PredictivePolicingAgent, PredictivePolicingAgentParams
from predict_policing_env import PredictivePolicingEnv
import predict_policing_env
from predict_hotspot_seppR_model import predict_hotspot_SEPP_R_model, get_bogota_mask
from predict_policing_simulation import *
import predict_policing_fairness_metrics 
import time

import sys
sys.path.insert(0, os.getcwd() + '/../')
import vanilla_monte_carlo

from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [4]:
# load data
import pickle
results = []
with open('simulation_results/10-15_w_utility.pkl', 'rb') as f:
    results = results + pickle.load(f)
results[-1]

(14,
 (True,
  16,
  0,
  [],
  2917.48570895195))

In [5]:
potential_hot_spot_discover_rate = [1.0, 0.95, 0.9, 0.85, 0.8]
potential_non_hot_spot_discover_rate = [0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2]
potential_n_hot_spots = [50]
potential_hot_spot_effect_size = [1, 2, 3]
potential_rel_num_gap_threshold = [0.30]
potential_fairness_window_size = [30]

# get all possible combinations of parameters
all_params = [potential_hot_spot_discover_rate, potential_non_hot_spot_discover_rate, potential_n_hot_spots,
                potential_hot_spot_effect_size, potential_rel_num_gap_threshold,
                potential_fairness_window_size]
all_params = np.array(np.meshgrid(*all_params)).T.reshape(-1, len(all_params))
print(all_params.shape)

(105, 6)


In [20]:
df_all_params = pd.DataFrame(columns=['hot_spot_discover_rate', 'non_hot_spot_discover_rate', 'hot_spot_effect_size',
                                       'avg_max_unfairness_rln', 'avg_mean_unfairness_rln', 'initial_unfairness_rln',
                                       'avg_utility_mean_discover_rate', 'avg_utility_total_discover_n', 'avg_n_correct_hotspot', 
                                       'n_traces'])
cur_config_index = [0, 5]
cur_results_file_path = 'vanilla_results/'+str(cur_config_index[0]).zfill(2)+'-'+\
    str(cur_config_index[1]).zfill(2)+'_w_utility.pkl'
with open(cur_results_file_path, 'rb') as f:
    cur_results = pickle.load(f)
for i in range(len(all_params)):
    if i not in range(cur_config_index[0], cur_config_index[1]):
        cur_config_index[0] += 5
        cur_config_index[1] += 5
        cur_results_file_path = 'vanilla_results/'+str(cur_config_index[0]).zfill(2)+'-'+\
            str(cur_config_index[1]).zfill(2)+'_w_utility.pkl'
        with open(cur_results_file_path, 'rb') as f:
            cur_results = pickle.load(f)
    offset_i = i - cur_config_index[0]
    cur_config = all_params[i]
    cur_result = cur_results[offset_i]

    hot_spot_discover_rate = cur_config[0]
    non_hot_spot_discover_rate = cur_config[1]
    hot_spot_effect_size = cur_config[3]

    max_unfairness_rln_list = []
    mean_unfairness_rln_list = []
    initial_unfairness_rln_list = []
    utility_mean_discover_rate_list = []
    utility_total_discover_n_list = []
    n_correct_hotspot_list = []

    for his in cur_result[1][2]:
        max_unfairness_rln_list.append(np.max(his.unfairness_list))
        mean_unfairness_rln_list.append(np.mean(his.unfairness_list))
        initial_unfairness_rln_list.append(his.unfairness_list[0])

        utility_mean_discover_rate_list.append(np.mean(his.incident_discover_rate_city))
        utility_total_discover_n_list.append(np.sum(his.incident_discover_num_city))
        n_correct_hotspot_list.append(np.mean(his.correct_hotspot_num_list))
    
    df_all_params.loc[i] = [hot_spot_discover_rate, non_hot_spot_discover_rate, hot_spot_effect_size,
                            np.mean(max_unfairness_rln_list), np.mean(mean_unfairness_rln_list), np.mean(initial_unfairness_rln_list),
                            np.mean(utility_mean_discover_rate_list), np.mean(utility_total_discover_n_list), np.mean(n_correct_hotspot_list),
                            cur_result[1][1]]
print(df_all_params.shape)

(105, 10)


In [22]:
df_all_params

,hot_spot_discover_rate,non_hot_spot_discover_rate,hot_spot_effect_size,avg_max_unfairness_rln,avg_mean_unfairness_rln,initial_unfairness_rln,avg_utility_mean_discover_rate,avg_utility_total_discover_n,avg_n_correct_hotspot,n_traces
0,1.0,0.50,1,0.269339,0.162631,0.007391,0.667727,33136.500000,15.935859,32
1,1.0,0.45,1,0.273956,0.172383,0.007391,0.632467,31389.562500,15.780938,16
2,1.0,0.40,1,0.297704,0.181402,0.007391,0.598123,29683.458333,15.666667,24
3,1.0,0.35,1,0.298790,0.189401,0.007391,0.562082,27896.687500,15.464844,16
4,1.0,0.30,1,0.331910,0.206556,0.007391,0.526772,26151.208333,15.248750,24
...,...,...,...,...,...,...,...,...,...,...
100,0.8,0.40,3,0.271024,0.151145,0.007391,0.674374,33462.000000,16.368750,16
101,0.8,0.35,3,0.276835,0.150085,0.007391,0.658732,32685.500000,16.366250,16
102,0.8,0.30,3,0.302366,0.157501,0.007391,0.638333,31670.000000,16.308333,24
103,0.8,0.25,3,0.337151,0.198417,0.007391,0.613274,30432.000000,16.230000,16


In [21]:
df_all_params['n_traces'] = df_all_params['n_traces'].astype(int)
df_all_params['hot_spot_effect_size'] = df_all_params['hot_spot_effect_size'].astype(int)

In [23]:
df_all_params.to_csv('simulation_results/all_params_w_utility105.csv')